## Requirement

(Optional) Using Generation by Syntax Tree and PAIP 151-167. Generate Sentence by Grammar. Implement a dialogue system, you may implement a mock system like West World, or Prison Break, etc.

- Using AIML markup language to generate same pattern. 

- Think what markup language or rule based can do, what they cannot do?

Bonus: If you finish this task, please connect with me. I will send you a **mysterious gift**.

### References:
a. PAIP https://github.com/Artificial-Intelligence-for-NLP/References/blob/master/AI%20%26%20Machine%20Learning/Paradigms-of-Artificial-Intelligence-Programming%20copy.pdf

b. https://www.wikiwand.com/en/ELIZA

c. https://www.eclecticenergies.com/ego/eliza

d. https://baike.baidu.com/item/%E8%89%BE%E4%B8%BD%E8%8E%8E/9030600

# chapter 5: ELIZA: Dialog with a Machine
## 5.1 Describing and Specifying ELIZA
Now that we have an idea of what ELIZA is like, we can begin the description and spec- ificationof the program, and eventually move to the implementation and debugging. The ELIZA algorithm can be described simply as: 

- (1)read an input, 

- (2) find a pattern that matches the input, 

- (3)transform the input into a response, and 

- (4)print the response. These four steps are repeated for each input.

The specificationand implementation of steps (1)and (4)are trivial: for (I),use the built-in read function to read a list of words, and for (4)use pri n t to print the list of words in the response.
Of course, there are some drawbacks to this specification. The user will have to type a real list- using parentheses- and the user can't use characters that are special to read, like quotation marks, commas, and periods. So our input won't be as unconstrained as in the sample dialog, but that's a small price to pay for the convenience of having half of the problem neatly solved.


## 5.2 Pattern Matching
```lisp
(defun simple-equal (x y)
  (if (or (atom x) (atom y))
	(eql x y)
	(and (simple-equal (car x) (car y))
		 (simple-equal (cdr x) (cdr y)))))

(defun pat-match (pattern input)
  (if (variable-p pattern)
	t
	(if (or (atom pattern ) (atom input))
	  (eql pattern input)
	  (and (pat-match (car pattern) (car input))
		   (pat-match (cdr pattern) (cdr input))))))
```

```lisp
(defun variable-p (x)
  (and (symbolp x) (equal (char (symbol-name x) 0) #\?)))

(defconstant fail nil)
(defconstant no-bindings '((t . t)))

(defun get-binding (var bindings)
  (assoc var bindings))

(defun binding-val (binding)
  (cdr binding))

(defun lookup (var bindings)
  (binding-val (get-binding var bindings)))

(defun extend-bindings (var val bindings)
  (cons (cons var val) bindings))

(defun pat-match (pattern input &optional (bindings no-bindings))
  (cond ((eq bindings fail) fail)
		((variable-p pattern)
		 (match-variable pattern input bindings))
		((eql pattern input) bindings)
		((and (consp pattern) (consp input))
		 (pat-match (cdr pattern) (cdr input)
					(pat-match (car pattern) (car input) bindings)))
		(t fail)))

(defun match-variable (var input bindings)
  (let ((binding (get-binding var bindings)))
	(cond ((not binding) (extend-bindings var input bindings))
		  ((equal input (binding-val binding)) bindings)
		  (t fail))))


(defun extend-bindings (var val bindings)
  (cons (cons var val)
		(if (eq bindings no-bindings)
		  nil
		  bindings)))

(defun pat-match (pattern input &optional (bindings no-bindings))
  (cond ((eq bindings fail) fail)
		((variable-p pattern) 
		 (match-variable pattern input bindings))
		((eql pattern input) bindings)
		((segment-pattern-p pattern)
		 (segment-match pattern input bindings))
		((and (consp pattern) (consp input))
		 (pat-match (cdr pattern) (cdr input)
					(pat-match (car pattern) (car input) bindings)))
		(t fail)))

(defun segment-pattern-p (pattern)
  (and (consp pattern)
	   (starts-with (car pattern) '?*)))

(defun starts-with (lst symb)
  (if (consp lst)
	(eql (car lst) symb)
	(eql lst symb)))

(defun segment-match (pattern input bindings &optional (start 0))
  (let ((var (cadr (car pattern)))
		(pat (cdr pattern)))
	(if (null pat)
	  (match-variable var input bindings)
	  (let ((pos (position (car pat) input :start start :test #'equal)))
		(if (null pos)
		  fail
		  (let ((b2 (pat-match pat (subseq input pos) bindings)))
			(if (eq b2 fail)
			  (segment-match pattern input bindings (1+ pos))
			  (match-variable var (subseq input 0 pos) b2))))))))

(defun segment-match (pattern input bindings &optional (start 0))
  (let ((var (cadr (car pattern)))
		(pat (cdr pattern)))
	(if (null pat)
	  (match-variable var input bindings)
	  (let ((pos (position (car pat) input :start start :test #'equal)))
		(if (null pos)
		  fail
		  (let ((b2 (pat-match
					  pat (subseq input pos)
					  (match-variable var (subseq input 0 pos)
									  bindings))))
			(if (eq b2 fail)
			  (segment-match pattern input bindings (1+ pos))
			  b2)))))))

(defun rule-pattern (rule) (car rule))
(defun rule-responses (rule) (cdr rule))

(defparameter *eliza-rules*
  '((((?* ?x) hello (?* ?y))
	 (how do you do. Please state your problem.))
	(((?* ?x) i want (?* ?y))
	 (what would it mean if you got ?y)
	 (why do you want ?y))
	(((?* ?x) if (?* ?y))
	 (do you really think its likely that ?y)
	 (what do you think about ?y)
	 (really-- if ?y))
	(((?* ?x) no (?* ?y))
	 (why not?)
	 (you are being a bit negative)
	 (are you saying "NO" just to be negative?))
	(((?* ?x) i feel (?* ?y))
	 (do you often feel ?y ?))
	(((?* ?x) i felt (?* ?y))
	 (what other feelings do you have?))))

(defun eliza ()
  (loop
	(print 'eliza>)
	(write (flattern (use-eliza-rule (read))) :pretty t)))

(defun use-eliza-rule (input)
  (some #'(lambda (rule)
			(let ((result (pat-match (rule-pattern rule) input)))
			  (if (not (eq result fail))
				(sublis (switch-viewpoint result)
						(random-elt (rule-responses rule))))))
		*eliza-rules*))
(defun switch-viewpoint (words)
  (sublis '((i . you) (you . i) (me . you) (am . are))
		  words))

(defun random-elt (lst)
  (elt lst (random (length lst))))

(defun mappend (fn lst)
  (apply #'append (mapcar fn lst)))

(defun flattern (lst)
  (mappend #'mklist lst))

(defun mklist (x)
  (if (listp x)
	x
	(list x)))

(eliza)
```